In [1]:
import tensorflow as tf
print(tf.__version__)

1.2.0


In [2]:
input_data = [[1,5,3,7,8,10,12],
              [5,8,10,3,9,7,1]]

label_data = [[0,0,0,1,0],
              [1,0,0,0,0]]

INPUT_SIZE = 7
HIDDEN1_SIZE = 10
HIDDEN2_SIZE = 8
CLASSES = 5

LEARNING_RATE = 0.01

In [3]:
"""
shape를 정해줄 때에서 법칙이 있는데 첫번째 디멘전이 있고, 두법째 디멘전이 있고, 세번째 디멘전이있고... 등등 쭉 나가는데
첫번째 디멘전은 무조건 배치가 들어간다 즉 입력되는 데이터가 몇개인가? 배치사이즈 
보통 배치에 개수는 알수 없는 경우가 일반적이다 즉 데이터가 몇개가 정확히 들어올지 모른다. 그래서 1차원 자리는 None로 보통 한다. 
"""

x = tf.placeholder(tf.float32, shape=[None, INPUT_SIZE]) # 보통 직접 숫자를 써주는 것이 아니라 상수를 이용한다. 
y_ = tf.placeholder(tf.float32, shape=[None, CLASSES])

In [4]:
tensor_map = {x: input_data, y_: label_data} #세션의 두번째 파라미터에 입력을 하는데 이것은 세션을 돌릴때 {x:,y_:}텐서는 위의 input_data와 label_data를 써라 라는 소리

위에까지로 보면 인풋데이터 정의 아웃풋데이터 정의 placeholder정의가 준비 되었다 이젠 모델을 설계 하면 된다. 


신경망은 인풋데이터들과 곱해질 weight가 필요 하다 

In [5]:
#우리는 예전에 variable를 사용할때 tf.Variable([1,2,3]) 이렇게 값을 직접 지정 했지만 실제 사용할때에는 랜덤값으로 한다. 
#tf.truncated_normal(shape)를 넣는데 이때 리턴되는 결과가 Variable의 shape로 들어간다. 
W_h1 = tf.Variable(tf.truncated_normal([INPUT_SIZE, HIDDEN1_SIZE]), dtype=tf.float32)

#bias라는 것도 중요하다 반드시 넣어줘서 high bias현상을 없애 줘야 한다.
b_h1 = tf.Variable(tf.zeros(shape=[HIDDEN1_SIZE]), dtype=tf.float32)

W_h2 = tf.Variable(tf.truncated_normal([HIDDEN1_SIZE, HIDDEN2_SIZE]), dtype=tf.float32)
b_h2 = tf.Variable(tf.zeros(shape=[HIDDEN2_SIZE]), dtype=tf.float32)

W_o = tf.Variable(tf.truncated_normal([HIDDEN2_SIZE, CLASSES]), dtype=tf.float32)
b_o = tf.Variable(tf.zeros(shape=[CLASSES]), dtype=tf.float32)


#세이브를 할경우 Variable가 끝나는 시점에 생성을 해준다. 
#saver = tf.train.Saver()  #이렇게 해주면 위에 나오는 Variable는 다 저장 된다.

param_list = [W_h1, b_h1, W_h2, b_h2, W_o, b_o]
saver = tf.train.Saver(param_list)


In [6]:
hidden1 = tf.sigmoid(tf.matmul(x, W_h1) + b_h1)
hidden2 = tf.sigmoid(tf.matmul(hidden1, W_h2) + b_h2)
y = tf.sigmoid(tf.matmul(hidden2, W_o) + b_o)

위에처럼 하면 모델 설계가 다 된 것이다. 

In [7]:
print(y)

Tensor("Sigmoid_2:0", shape=(?, 5), dtype=float32)


트레이닝을 시키기 위해서는 cost function이 필요하게 된다.

In [8]:
#크로스 엔트로피!
cost = tf.reduce_mean(tf.reduce_sum(-y_ * tf.log(y) - (1-y_) * tf.log(1-y), reduction_indices=1))  # y_이 0일경우에는 (1-y_)*tf.log(1-y) 가 돌게 한다. 
train = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE).minimize(cost)


In [13]:
comp_pred = tf.equal(tf.arg_max(y, 1), tf.arg_max(y_, 1))
accurcy = tf.reduce_mean(tf.cast(comp_pred, dtype=tf.float32))

In [14]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(2000):
    _, cost_val, acc = sess.run([train, cost, accurcy], feed_dict=tensor_map)  #여기서 train과 cost는 따로 실행이 된다. 서로 연관된게 아니다. 
    if i % 100 == 0:
        saver.save(sess, "./tenserflow_live.ckpt")
        print("step: ", i, " loss: ", cost_val, " acc: ", acc)

sess.close()
    

step:  0  loss:  6.29402  acc:  0.0
step:  100  loss:  2.52943  acc:  0.5
step:  200  loss:  1.83996  acc:  0.5
step:  300  loss:  1.55292  acc:  1.0
step:  400  loss:  1.37556  acc:  1.0
step:  500  loss:  1.23686  acc:  1.0
step:  600  loss:  1.11499  acc:  1.0
step:  700  loss:  1.00496  acc:  1.0
step:  800  loss:  0.906505  acc:  1.0
step:  900  loss:  0.819279  acc:  1.0
step:  1000  loss:  0.742331  acc:  1.0
step:  1100  loss:  0.674484  acc:  1.0
step:  1200  loss:  0.614614  acc:  1.0
step:  1300  loss:  0.561725  acc:  1.0
step:  1400  loss:  0.51495  acc:  1.0
step:  1500  loss:  0.473529  acc:  1.0
step:  1600  loss:  0.436792  acc:  1.0
step:  1700  loss:  0.40415  acc:  1.0
step:  1800  loss:  0.375085  acc:  1.0
step:  1900  loss:  0.349142  acc:  1.0
